In [15]:
import pandas as pd
import requests

In [16]:
#install for sentiment analysis
!pip install textblob

In [17]:
#install corpora
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!

In [19]:
# Read location info and listings CSV files into pandas DataFrames
seattle_listings = pd.read_csv('http://data.insideairbnb.com/united-states/wa/seattle/2023-09-18/visualisations/listings.csv')

In [20]:
seattle_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,6606,Guesthouse in Seattle · ★4.60 · 1 bedroom · 1 ...,14942,Joyce,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,30,160,2023-08-05,0.93,2,177,1,str-opli-19-002622
1,9419,Rental unit in Seattle · ★4.72 · 1 bedroom · 1...,30559,Angielena,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,85,2,193,2023-09-04,1.21,9,365,23,STR-OPLI-19-003039
2,9531,Home in Seattle · ★4.96 · 2 bedrooms · 3 beds ...,31481,Cassie,West Seattle,Fairmount Park,47.55495,-122.38663,Entire home/apt,185,3,77,2023-09-09,0.54,2,318,12,STR-OPLI-19-002182
3,9534,Guest suite in Seattle · ★4.99 · 2 bedrooms · ...,31481,Cassie,West Seattle,Fairmount Park,47.55627,-122.38607,Entire home/apt,155,2,75,2023-05-28,0.53,2,230,7,STR-OPLI-19-002182
4,9596,Rental unit in Seattle · ★4.56 · 1 bedroom · 4...,14942,Joyce,Other neighborhoods,Wallingford,47.65608,-122.33602,Entire home/apt,130,30,97,2020-09-28,0.65,2,0,0,STR -OPLI-19-002622


In [22]:
#using gzip to retrieve the files

import requests
import gzip
import io

# URL of the reviews.csv.gz file from Inside Airbnb for Seattle
url = 'http://data.insideairbnb.com/united-states/wa/seattle/2023-09-18/data/reviews.csv.gz'

# Download the file
response = requests.get(url, stream=True)

if response.status_code == 200:
    # Create a file-like object for the downloaded content
    compressed_file = io.BytesIO(response.content)

    # Extract the gzipped content
    with gzip.GzipFile(fileobj=compressed_file, mode='rb') as gz_file:
        # Read the CSV content from the extracted file
        with io.TextIOWrapper(gz_file, encoding='utf-8') as file:
            # Create a DataFrame from the CSV content
            df = pd.read_csv(file)

    # Display the first few rows of the DataFrame
    print(df.head())
else:
    print("Failed to download the file")

   listing_id       id        date  reviewer_id    reviewer_name  \
0        6606     5664  2009-07-17        18085           Vivian   
1      353110  1080517  2012-04-03      2019171  Allison & Jerry   
2      353110  1604096  2012-07-01      2086430             Mark   
3      353110  1904384  2012-08-06      2886945           Davida   
4      353110  2563507  2012-10-09      3619132            Trish   

                                            comments  
0  The Urban Cottage is comfortable, beautiful, f...  
1  Simply a lovely place to stay.  Quite, private...  
2  My girlfriend and I recently moved to Seattle ...  
3  This is a great place in a great location. The...  
4  My daughter and I stayed at Eric's place last ...  


In [24]:
#using gzip in a function
import requests
import gzip
import io

def fetch_csv_gz_from_url(url, dataframe_name):
    try:
        # Download the file
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            # Create a file-like object for the downloaded content
            compressed_file = io.BytesIO(response.content)

            # Extract the gzipped content
            with gzip.GzipFile(fileobj=compressed_file, mode='rb') as gz_file:
                # Read the CSV content from the extracted file
                with io.TextIOWrapper(gz_file, encoding='utf-8') as file:
                    # Create a DataFrame from the CSV content
                    df = pd.read_csv(file)

            # Assign a specified name to the DataFrame
            globals()[dataframe_name] = df
            print(f"{dataframe_name} DataFrame created successfully")
            return df
        else:
            print("Failed to download the file")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage:
url = 'http://data.insideairbnb.com/united-states/wa/seattle/2023-09-18/data/reviews.csv.gz'
resulting_dataframe = fetch_csv_gz_from_url(url, 'seattle_reviews_df')
if resulting_dataframe is not None:
    print(resulting_dataframe.head())

seattle_reviews_df DataFrame created successfully
   listing_id       id        date  reviewer_id    reviewer_name  \
0        6606     5664  2009-07-17        18085           Vivian   
1      353110  1080517  2012-04-03      2019171  Allison & Jerry   
2      353110  1604096  2012-07-01      2086430             Mark   
3      353110  1904384  2012-08-06      2886945           Davida   
4      353110  2563507  2012-10-09      3619132            Trish   

                                            comments  
0  The Urban Cottage is comfortable, beautiful, f...  
1  Simply a lovely place to stay.  Quite, private...  
2  My girlfriend and I recently moved to Seattle ...  
3  This is a great place in a great location. The...  
4  My daughter and I stayed at Eric's place last ...  


In [25]:
seattle_reviews_df.dtypes

listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object

In [28]:
#Filter seattle reviews for 2022 and 2023 only
seattle_reviews_df['date'] = pd.to_datetime(seattle_reviews_df['date'],format='%Y-%m-%d')
seattle_reviews_recent = seattle_reviews_df[(seattle_reviews_df['date'].dt.year == 2022) | (seattle_reviews_df['date'].dt.year == 2023)]
seattle_reviews_recent.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
138,353110,573999724723704695,2022-03-02,132891274,Nick,Stayed for a month while relocating to Seattle...
139,353110,596369993109024929,2022-04-02,404948475,Genna,The West Seattle Backyard Cottage is in beauti...
148,353110,639180709801600503,2022-05-31,439438613,Lindsey,Location was fantastic! Eric was great and acc...
149,353110,661713603181951779,2022-07-01,52541293,Jennifer,My husband and I stayed at West Seattle Cottag...
150,353110,697957315562343313,2022-08-20,436930794,Ashley,I stayed at the cottage for 7 weeks. It's in a...


In [10]:
from io import BytesIO

def read_csv_gz_from_url(url):
    """
    Fetches a gzipped CSV file from a URL, decompresses it, and reads it into a Pandas DataFrame.

    Args:
    - url: URL of the gzipped CSV file

    Returns:
    - Pandas DataFrame containing the data from the CSV file
    """
    response = requests.get(url)
    compressed_file = BytesIO(response.content)
    
    with gzip.open(compressed_file, 'rb') as gz_file:
        df = pd.read_csv(gz_file)
    
    return df

# Read data from each URL and assign to different DataFrame names
seattle_reviews = read_csv_gz_from_url('http://data.insideairbnb.com/united-states/wa/seattle/2023-09-18/data/reviews.csv.gz') 
la_reviews = read_csv_gz_from_url('http://data.insideairbnb.com/united-states/ca/los-angeles/2023-09-03/data/reviews.csv.gz') 

In [12]:
from textblob import TextBlob

#this can take a bit since TextBlob processing each text entry individually 
def add_sentiment_column(df):
    """
    Apply TextBlob's sentiment analysis to a specified text column in a DataFrame
    and create a new column for sentiment scores.

    Returns:
    - Updated DataFrame with the new column for sentiment scores
    """
    df['sentiment_score'] = df['comments'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
    return df

seattle_reviews = add_sentiment_column(seattle_reviews)

# Display the updated DataFrame with sentiment scores
seattle_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment_score
0,6606,5664,2009-07-17,18085,Vivian,"The Urban Cottage is comfortable, beautiful, f...",0.330833
1,353110,1080517,2012-04-03,2019171,Allison & Jerry,"Simply a lovely place to stay. Quite, private...",0.253125
2,353110,1604096,2012-07-01,2086430,Mark,My girlfriend and I recently moved to Seattle ...,0.299667
3,353110,1904384,2012-08-06,2886945,Davida,This is a great place in a great location. The...,0.383300
4,353110,2563507,2012-10-09,3619132,Trish,My daughter and I stayed at Eric's place last ...,0.377778


In [ ]:
import urllib.request
import gzip
import csv
import zipfile
import io

# Replace 'your_url_here.csv.gz' with the actual URL of the file you want to download
url = 'your_url_here.csv.gz'

# Download the file
response = urllib.request.urlopen(url)
compressed_file = io.BytesIO(response.read())

# Extract the gzipped content
with gzip.GzipFile(fileobj=compressed_file, mode='rb') as gz_file:
    # Read the CSV content from the extracted file
    with io.TextIOWrapper(gz_file, encoding='utf-8') as file:
        csv_content = csv.reader(file)
        
        # Process the CSV data (example: printing the content)
        for row in csv_content:
            print(row)  # Do whatever processing you need here
